# Remoção condicional de linhas de um dataset - Implantação

A partir de uma condição, remove as linhas que satisfazem uma condição fornecida.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import pandas as pd
import logging

class Model:
    def __init__(self):
        self.loaded = False

    def load(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/remove-rows.joblib")
        self.columns = artifacts["columns"]
        self.filter_query = artifacts["filter_query"]
        self.colum_types = artifacts["colum_types"]

        self.loaded = True

    def class_names(self):
        return self.columns

    def predict(self, X, feature_names, meta=None):

        if not self.loaded:
            self.load()

        # Transforma X em um dataframe
        df = pd.DataFrame(data=X, columns=feature_names)

        # Reordena colunas para seguirem o padrão do dataset original
        df = df[self.columns]

        # Transformando tipos numéricos recebidos como não-numéricos
        for colum, colum_type in zip(self.columns, self.colum_types):
            
            # Verificando se tipo é numérico
            if pd.api.types.is_numeric_dtype(colum_type):
                df[colum] = pd.to_numeric(df[colum])

        # Aplica a query de filtro
        df = df.query(self.filter_query)

        # logging.error(df)
        # assert False

        return df.to_numpy()